In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [48]:
torch.manual_seed(1337)

In [49]:
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

In [50]:
masked_tril = torch.tril(torch.ones(T, T))
w_t = torch.zeros(masked_tril.shape)
w_t = w_t.masked_fill(masked_tril==0, float('-inf'))
w_t = F.softmax(w_t, dim=-1)

In [51]:
print(w_t)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


In [52]:
out = w_t @ x

In [53]:
print(out.shape)

torch.Size([4, 8, 32])


In [54]:
# Add self-attention block
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = key(x) # (B, T, 16)

In [55]:
w_t = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) => (B, T, T)
# Use scaling to control variance at initialization to aovid the vector is shapened to a specific value
w_t = w_t * C**-0.5

In [56]:
masked_tril = torch.tril(torch.ones(T, T))
w_t = w_t.masked_fill(masked_tril==0, float('-inf'))
w_t = F.softmax(w_t, dim=-1)

In [57]:
print(w_t[0])

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3736, 0.6264, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2564, 0.2712, 0.4723, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1143, 0.1323, 0.2391, 0.5143, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1326, 0.2242, 0.1407, 0.0555, 0.4470, 0.0000, 0.0000, 0.0000],
        [0.1069, 0.0977, 0.1042, 0.2037, 0.0551, 0.4324, 0.0000, 0.0000],
        [0.1024, 0.1390, 0.1443, 0.1308, 0.1560, 0.1345, 0.1929, 0.0000],
        [0.1201, 0.1376, 0.1187, 0.1044, 0.1266, 0.1022, 0.1142, 0.1762]],
       grad_fn=<SelectBackward0>)


In [58]:
value = nn.Linear(C, head_size, bias=False)
v = value(x)

In [59]:
out = w_t @ v
print(out.shape)

torch.Size([4, 8, 16])
